### Считываем данные (refined dataset)

In [1]:
import array, struct, sys, os, tqdm
import numpy as np

def read_binaries(path_binfiles):
    result = {}
    #cnt = 0
    for binfile in tqdm.tqdm(os.listdir(path_binfiles)):
        if (binfile.split('.')[1] != 'bin'):
            continue
        pdbcode = binfile.split('.')[0]                     # name of file (pdbcode)
        F = open('{0}/{1}'.format(path_binfiles, binfile), 'rb')
        n_decoys = struct.unpack('i', F.read(4))[0]         # number of decoys (=19 for this dataset)
        dimension = struct.unpack('i', F.read(4))[0]        # data dimensionality (23 protein types x 40 ligand types x 7 bins for this dataset)
        res = []
        for i in range(n_decoys):
            label = struct.unpack('d', F.read(8))[0]        # label (1 for native, -1 for non-native)
            data = array.array('d')                         
            data.fromfile(F, dimension)                     # feature vector (histograms, can be represented as a 23x40x7 matrix) 
            res.append([label, data])
        result[pdbcode] = res
        F.close()
        #if cnt == 13080:
        #    break
        #cnt += 1
    return result

In [2]:
result = read_binaries('../../../../../basic_experiment/general-no2013_t14_t3_l7.0_g1.0_r1.0')

100%|██████████| 13090/13090 [01:40<00:00, 129.63it/s]


In [3]:
with open('../../../../../basic_experiment/affinity_data_refined.csv', 'r') as f:
    data = f.read().split('\n')
    data = data[1:-1]

In [4]:
datasets = [
    {'name': d.split(',')[0], 'value': d.split(',')[1], 'type': d.split(',')[3]}
    for d in data
]

In [5]:
Kd_values = []
Ki_values = []
for d in datasets:
    if d['type'] == 'Kd':
        Kd_values.append(d)
    else:
        Ki_values.append(d)

In [6]:
Kd_data = []
for item in Kd_values:
    Kd_data.append([item['value']] + result[item['name']])

In [7]:
Ki_data = []
for item in Ki_values:
    if item['name'] != '966c':
        Ki_data.append([item['value']] + result[item['name']])

### 1 этап. Предсказание свободной энергии.
Для обучения рассматриваем для каждого комплекса только его нативную позу (т.к. только для них известны значения свободной энергии).

Берем все нативные позы со значениями Ki (Ki_data) из refined dataset.

Предсказываем значение Ki.

### Работа с данными:

    1) Разделение данных на test и train и выделение аффинных данных (X_nat_train)
    
    2) Замена переменных
    
    3) Запись в файл нового вектора X (для работы в liblinear)

In [8]:
import time
import numpy as np
from math import log, exp
from scipy.linalg import sqrtm, inv, norm
from scipy.optimize import minimize

In [9]:
start_time = time.time()
data = Ki_data
train = data[:int(len(data) * 0.6)]
test = data[int(len(data) * 0.6):]

# Матрица признаков (для которых аффинности известны)
X_nat_train = np.matrix([
    t[1][1]
    for t in train
]).T

# Столбец значений свободной энергии
s_train = np.matrix([
    float(t[0])
    for t in train
]).T
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.4665520191192627 seconds ---


In [10]:
print(X_nat_train.shape)
print(s_train.shape)

(6440, 1245)
(1245, 1)


In [11]:
start_time = time.time()
X_train = []
for t in train:
    for pose in t[1:]:
        X_train.append(pose[1])
        
X_train = np.matrix(X_train).T

y_train = []
for t in train:
    for pose in t[1:]:
        y_train.append(pose[0])

y_train = np.matrix(y_train).T
print("--- %s seconds ---" % (time.time() - start_time))

--- 19.268756866455078 seconds ---


In [12]:
start_time = time.time()
X_test = []
for t in test:
    for pose in t[1:]:
        X_test.append(pose[1])
        
X_test = np.matrix(X_test).T

y_test = []
for t in test:
    for pose in t[1:]:
        y_test.append(pose[0])

y_test = np.matrix(y_test).T
print("--- %s seconds ---" % (time.time() - start_time))

--- 14.379429817199707 seconds ---


In [13]:
print(X_train.shape)
print(y_train.shape)

(6440, 23655)
(23655, 1)


In [84]:
# Замена переменных
start_time = time.time()
Cr = 100 # Коэффициент регуляризации
XXT = X_nat_train @ X_nat_train.T
I = np.identity(XXT.shape[0])
A = np.real(sqrtm(0.5 * I + Cr * XXT))
print("--- %s seconds ---" % (time.time() - start_time))

--- 299.31849813461304 seconds ---


In [85]:
start_time = time.time()
A_inv = inv(A)
B = Cr * A_inv @ X_nat_train @ s_train
print("--- %s seconds ---" % (time.time() - start_time))

--- 13.844375133514404 seconds ---


In [86]:
print(A.shape)
print(B.shape)

(6440, 6440)
(6440, 1)


Заменяем матрицу $X$ на $(A^{-1})^{\text{T}}X$ и записываем ее вместе с вектором $y$ в файл, чтобы подать этот файл в liblinear.
А также вектор постоянных слагаемых внутри функции потерь.

In [87]:
newX = (A_inv.T @ X_train).T
print(newX.shape)

(23655, 6440)


In [88]:
with open("ki_train", "w") as f:
    for i in tqdm.tqdm(range(newX.shape[0])):
        y_i = ("+1 " if y_train[i] == 1 else "-1 ")
        f.write(y_i)
        for j in range(newX.shape[1]):
            f.write(str(j + 1) + ":" + str(newX[i,j]) + " ")
        f.write("\n")

100%|██████████| 23655/23655 [13:32<00:00, 29.13it/s]


In [89]:
constant = np.multiply(y_train, ((A_inv @ B).T @ X_train).T)
print(constant.shape)

(23655, 1)


In [90]:
float(constant[0])

8.619652833637915

In [91]:
with open("ki_train_constant", "w") as f:
    for i in tqdm.tqdm(range(constant.shape[0])):
        f.write(str(float(constant[i])) + "\n")

100%|██████████| 23655/23655 [00:00<00:00, 39954.73it/s]


Тестовые файлы (проверка правильно угаданных нативных поз)

In [140]:
newX_test = (A_inv.T @ X_test).T
print(newX.shape)

(23655, 6440)


In [93]:
with open("ki_test", "w") as f:
    for i in tqdm.tqdm(range(newX_test.shape[0])):
        y_i = ("+1 " if y_test[i] == 1 else "-1 ")
        f.write(y_i)
        for j in range(newX_test.shape[1]):
            f.write(str(j + 1) + ":" + str(newX_test[i,j]) + " ")
        f.write("\n")

100%|██████████| 1662/1662 [00:57<00:00, 28.86it/s]


### Строим модель в liblinear, достаем вектор $w$ из файла .model и тестируем модель.

In [149]:
from scipy.stats import spearmanr, pearsonr
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [150]:
with open("../ki_train.model", "r") as f:
    data = f.read().split("\n")

In [151]:
newW = np.array(data[6:-1], dtype=float).reshape((6440, 1))

In [152]:
w = A_inv @ (newW + B)

In [153]:
np.savetxt("ki_w.txt", w.T)

In [154]:
# Матрица признаков (для которых аффинности известны)
X_test = np.matrix([
    t[1][1]
    for t in test
]).T

# Столбец значений свободной энергии
s_test = np.matrix([
    float(t[0])
    for t in test
]).T

In [155]:
prediction = w.T @ X_test
print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

Spearman:  SpearmanrResult(correlation=0.64238494002903612, pvalue=6.9057871865105226e-98)
Pearson:  (0.62790831694860949, 2.5052835343899646e-92)
R2:  -0.56026556888
MSE:  6.99550820623


In [45]:
prediction = w.T @ X_test
print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

Spearman:  SpearmanrResult(correlation=0.69166226720519497, pvalue=2.7519435180374767e-119)
Pearson:  (0.68399876227528322, 1.1131347687760896e-115)
R2:  0.349571970825
MSE:  2.91621805057


In [38]:
prediction = w.T @ X_test
print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

Spearman:  SpearmanrResult(correlation=0.69015191946932408, pvalue=1.4434995449354607e-118)
Pearson:  (0.68219302714172125, 7.5909374048635784e-115)
R2:  0.343886262877
MSE:  2.9417101318


In [31]:
prediction = w.T @ X_test
print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

Spearman:  SpearmanrResult(correlation=0.68356234628229917, pvalue=1.7725783590839373e-115)
Pearson:  (0.67049209677989263, 1.3800783330340288e-109)
R2:  -0.146462513968
MSE:  5.14020695231


In [75]:
#L2R_L1LOSS_SVC_DUAL
prediction = w.T @ X_test
print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

Spearman:  SpearmanrResult(correlation=0.69166226720519497, pvalue=2.7519435180374767e-119)
Pearson:  (0.68399876227528322, 1.1131347687760896e-115)
R2:  0.349571970825
MSE:  2.91621805057


In [30]:
# L2_LR с добавкой Constant
prediction = w.T @ X_test
print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

Spearman:  SpearmanrResult(correlation=0.69015191946932408, pvalue=1.4434995449354607e-118)
Pearson:  (0.68219302714172125, 7.5909374048635784e-115)
R2:  0.343886262877
MSE:  2.9417101318


In [40]:
# L2_LR с добавкой Constant
prediction = w.T @ X_test
print(spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

SpearmanrResult(correlation=0.69015191946932408, pvalue=1.4434995449354607e-118)
(0.68219302714172125, 7.5909374048635784e-115)
0.343886262877
2.9417101318


In [34]:
# L2_LOSS_SVC_DUAL
prediction = w.T @ X_test
print(spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

SpearmanrResult(correlation=0.69322426613187826, pvalue=4.9040465422768512e-120)
(0.68575165540911154, 1.7036531159905999e-116)
0.350186214271
2.91346406744


In [26]:
# Тренинговая аффинная выборка и Cr = 100 (вместо 10000), C = 1024
prediction = w.T @ X_test
print(spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

SpearmanrResult(correlation=0.68356234628229917, pvalue=1.7725783590839373e-115)
(0.67049209677989263, 1.3800783330340288e-109)
-0.146462513968
5.14020695231


In [23]:
# Тренинговая аффинная выборка и Cr = 10000 (вместо 100), C = 1024
prediction = w.T @ X_test_reg
print(spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

SpearmanrResult(correlation=0.62014223685389513, pvalue=2.6774230060625381e-133)
(0.60320970317061684, 2.4116978508802566e-124)
-0.209361431428
5.41602792232


In [76]:
# Добавил полную аффинную выборку и Cr = 100 (вместо 10), C = 1024
prediction = w.T @ X_test_reg
print(spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

SpearmanrResult(correlation=0.78110546631570565, pvalue=1.0389205936255559e-256)
(0.77570934211099696, 6.0031501273757858e-251)
0.327181243734
3.01316469659


In [65]:
# Добавил полную аффинную выборку и Cr = 5 (вместо 0.5), C = 1024
prediction = w.T @ X_test_reg
print(spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

SpearmanrResult(correlation=0.68905982191288717, pvalue=3.597549765965038e-176)
(0.67739176428319892, 4.3851607724117059e-168)
-0.233452411741
5.52391744043


In [47]:
# Аффинная выборка только из тренинговой выборки, Cr = 0.5, C = 1024
prediction = w.T @ X_test_reg
print(spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print(r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print(mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

SpearmanrResult(correlation=0.59782292604876919, pvalue=1.3171390817460266e-121)
(0.5853307218911773, 1.8685088214099835e-115)
-1.06153063991
9.23239919705


## Accuracy

In [156]:
pred_labels = newW.T @ newX_test.T

In [157]:
pred_labels = pred_labels.tolist()[0]

In [158]:
pl = [
    1 if p > 0 else -1
    for p in pred_labels
]

In [159]:
cnt = 0
for i in range(831):
    flag = True
    for j in range(19):
        index = i * 19 + j
        if (pl[index] != y_test[index]):
            flag = False
    if flag is True:
        cnt += 1
cnt / 831

0.5258724428399518

In [147]:
len(pl) / 19

831.0

# CASF-test

In [100]:
import os, glob, tqdm, struct, array
import numpy as np
from scipy.stats import pearsonr as pearsonr
import getpass 
username = getpass.getuser()
PATH_data = '/Users/igasov_ilya/Desktop/6semestr/Article/basic_experiment'

In [101]:
affinities195 = {}
for ki in Ki_values[int(len(Ki_data) * 0.6):]:
    affinities195[ki['name']] = float(ki['value'])

In [106]:
# load data for the docking test 
PATH_CASF = '{0}/'.format(PATH_data)
PATH_docking_d = '{0}general-no2013_t14_t3_l7.0_g1.0_r1.0/'.format(PATH_CASF)
dock_binaries = affinities195.keys()
recs195 = set([ele[:4] for ele in dock_binaries])
docking_dict = {rec: [] for rec in recs195}
docking_dict_qual = {}
docking_scores = {}
native_scores = {}
# мой скоринг-вектор:
#w = np.loadtxt('/home/maria/data/pdbbind/2016/protlig_stat/gen_t14_t3_l7.0_g1.0_r1.0/gen_t14_t3_l7.0_g1.0_r1.0_c20000_f500.txt')[1:]
w = np.loadtxt("tests/ki_w.txt")   
# flexibilities = {flex.split(',')[0]: float(flex.split(',')[1]) for flex in np.loadtxt('{0}casf-2013.flexibilities.csv'.format(PATH_CASF), dtype=np.str)}

# for irec, rec in enumerate(recs195):
for rec in tqdm.tqdm(recs195):
    # !!! uncomment solvent-related stuff if you need it !!!
    data = []
    F = open('{0}{1}.bin'.format(PATH_docking_d, rec), 'rb')
    n_decoys = struct.unpack('i', F.read(4))[0]         
    dimension = struct.unpack('i', F.read(4))[0]
    # F_sol = open('{0}{1}.bin.solvent'.format(PATH_docking_d, rec), 'rb')
    # n_decoys_sol = struct.unpack('i', F_sol.read(4))[0]
    # dimension_sol = struct.unpack('i', F_sol.read(4))[0]
    docking_scores[rec] = []
    docking_dict_qual[rec] = []
    for i in range(n_decoys):
        #rms = struct.unpack('d', F.read(8))[0]
        label = struct.unpack('d', F.read(8))[0]        # label (0 for native, 1 for RMSD < 1, 2 for RMSD < 2; 3 for RMSD < 3, -1 for RMSD > 3)
        data = array.array('d')
        data.fromfile(F, dimension)                     # feature vector: histograms (can be represented as a 23x40x7 matrix)
        # data_sol = array.array('d')
        # data_sol.fromfile(F_sol, dimension_sol)
        # data = list(data) + list(data_sol) + [flexibilities[rec]]
        score = np.sum(w * data)                          # !!! compute score with your scoring vector here !!!
        docking_scores[rec].append(score)
        if not label:
            native_scores[rec] = score
        docking_dict_qual[rec].append(label)
    F.close()
    # F_sol.close()

100%|██████████| 832/832 [00:05<00:00, 144.97it/s]


In [107]:
def run_docking_test():
    tq0, tq1, tq2, tq3 = [0]*3, [0]*3, [0]*3, [0]*3 
    
    tnq1, tnq2, tnq3 = [0]*3, [0]*3, [0]*3   # счетчики, не учитывющие нативные позы
    
    nPerc1, nPerc5, nPerc10 = 0, 0, 0
    
    for i_r, rec in enumerate(recs195):
        cur_best_res = docking_scores[rec]
        idx_sorted = np.argsort(cur_best_res)[::-1]
        q0, q1, q2, q3 = 0, 0, 0, 0
        nq1, nq2, nq3 = 0, 0, 0
        for k in range(3):
            if docking_dict_qual[rec][idx_sorted[k]] == 0:
                q0 = 1
                q1 = 1
                q2 = 1
                q3 = 1
            if docking_dict_qual[rec][idx_sorted[k]] == 1:
                q1 = 1
                q2 = 1
                q3 = 1
            if docking_dict_qual[rec][idx_sorted[k]] == 2:
                q2 = 1
                q3 = 1
            if docking_dict_qual[rec][idx_sorted[k]] == 3:
                q3 = 1
            tq0[k] += q0
            tq1[k] += q1
            tq2[k] += q2
            tq3[k] += q3
        j, k = 0, 0
        for j in range(5):
            if k >= 3:
                break
            if docking_dict_qual[rec][idx_sorted[j]] == 0:
                continue
            if docking_dict_qual[rec][idx_sorted[j]] == 1:
                nq1 = 1
                nq2 = 1
                nq3 = 1
            if docking_dict_qual[rec][idx_sorted[j]] == 2:
                nq2 = 1
                nq3 = 1
            if docking_dict_qual[rec][idx_sorted[j]] == 3:
                nq3 = 1
            tnq1[k] += nq1
            tnq2[k] += nq2
            tnq3[k] += nq3
            k += 1
        
    for k in range(3):
        print("-----------------------------------------------------------------------")
        print("Top{0:2d}_Native: {1:<5d} Top{0:2d}_Quality_1: {2:<5d} Top{0:2d}_Quality_2: {3:<5d} Top{0:2d}_Quality_3: {4:<5d} ".format(k+1, tq0[k], tq1[k], tq2[k], tq3[k]), end=' ')
        print("Natives excluded Top{0:2d}_Quality_1: {1:<5d} Top{0:2d}_Quality_2: {2:<5d} Top{0:2d}_Quality_3: {3:<5d}".format(k+1, tnq1[k], tnq2[k], tnq3[k]))
        print("-----------------------------------------------------------------------")
    for k in range(3):
        print("-----------------------------------------------------------------------")
        print("Top{0:2d}_Native: {1:<7.3f} Top{0:2d}_Quality_1: {2:<7.3f} Top{0:2d}_Quality_2: {3:<7.3f} Top{0:2d}_Quality_3: {4:<7.3f} ".format(k+1, tq0[k]/195, tq1[k]/195, tq2[k]/195, tq3[k]/195), end=' ')
        print("Natives excluded Top{0:2d}_Quality_1: {1:<7.3f} Top{0:2d}_Quality_2: {2:<7.3f} Top{0:2d}_Quality_3: {3:<7.3f}".format(k+1, tnq1[k]/195, tnq2[k]/195, tnq3[k]/195))
        print("-----------------------------------------------------------------------")     

In [108]:
def run_scoring_test():
    # Тут просто считаем корреляцию
    to_compare = []
    for com in affinities195.keys():
        to_compare.append([native_scores[com], affinities195[com]])
    to_compare = np.array(to_compare)
    print(pearsonr(to_compare[:, 0], to_compare[:, 1]))

In [109]:
run_docking_test()
run_scoring_test()

-----------------------------------------------------------------------
Top 1_Native: 0     Top 1_Quality_1: 128   Top 1_Quality_2: 128   Top 1_Quality_3: 128    Natives excluded Top 1_Quality_1: 128   Top 1_Quality_2: 128   Top 1_Quality_3: 128  
-----------------------------------------------------------------------
-----------------------------------------------------------------------
Top 2_Native: 0     Top 2_Quality_1: 259   Top 2_Quality_2: 259   Top 2_Quality_3: 259    Natives excluded Top 2_Quality_1: 259   Top 2_Quality_2: 259   Top 2_Quality_3: 259  
-----------------------------------------------------------------------
-----------------------------------------------------------------------
Top 3_Native: 0     Top 3_Quality_1: 388   Top 3_Quality_2: 388   Top 3_Quality_3: 388    Natives excluded Top 3_Quality_1: 388   Top 3_Quality_2: 388   Top 3_Quality_3: 388  
-----------------------------------------------------------------------
----------------------------------------

KeyError: '4cws'

## Предсказываем Kd + Ki вместе

In [47]:
data = []
for item in datasets:
    if item['name'] != '966c':
        data.append([item['value']] + result[item['name']])

In [48]:
start_time = time.time()
train = data[:int(len(data) * 0.6)]
test = data[int(len(data) * 0.6):]

# Матрица признаков (для которых аффинности известны)
X_nat_train = np.matrix([
    t[1][1]
    for t in train
]).T

# Столбец значений свободной энергии
s_train = np.matrix([
    float(t[0])
    for t in train
]).T
print("--- %s seconds ---" % (time.time() - start_time))

--- 4.490758895874023 seconds ---


In [49]:
start_time = time.time()
X_train = []
for t in train:
    for pose in t[1:]:
        X_train.append(pose[1])
        
X_train = np.matrix(X_train).T

y_train = []
for t in train:
    for pose in t[1:]:
        y_train.append(pose[0])

y_train = np.matrix(y_train).T
print("--- %s seconds ---" % (time.time() - start_time))

--- 37.7387900352478 seconds ---


In [50]:
print(X_nat_train.shape)
print(s_train.shape)

(6440, 2319)
(2319, 1)


In [51]:
print(X_train.shape)
print(y_train.shape)

(6440, 44061)
(44061, 1)


In [53]:
# Замена переменных
start_time = time.time()
Cr = 100 # Коэффициент регуляризации
XXT = X_nat_train @ X_nat_train.T
I = np.identity(XXT.shape[0])
A = np.real(sqrtm(0.5 * I + Cr * XXT))
print("--- %s seconds ---" % (time.time() - start_time))

--- 255.46882796287537 seconds ---


In [54]:
start_time = time.time()
A_inv = inv(A)
B = Cr * A_inv @ X_nat_train @ s_train
print("--- %s seconds ---" % (time.time() - start_time))

--- 17.51669406890869 seconds ---


In [55]:
print(A.shape)
print(B.shape)

(6440, 6440)
(6440, 1)


In [56]:
newX = (A_inv.T @ X_train).T
print(newX.shape)

(44061, 6440)


In [57]:
with open("kd_ki_train", "w") as f:
    for i in tqdm.tqdm(range(newX.shape[0])):
        y_i = ("+1 " if y_train[i] == 1 else "-1 ")
        f.write(y_i)
        for j in range(newX.shape[1]):
            f.write(str(j + 1) + ":" + str(newX[i,j]) + " ")
        f.write("\n")

100%|██████████| 44061/44061 [22:41<00:00, 32.37it/s]


In [58]:
constant = np.multiply(y_train, ((A_inv @ B).T @ X_train).T)
print(constant.shape)

(44061, 1)


In [59]:
float(constant[0])

8.379578518520287

In [60]:
with open("kd_ki_train_constant", "w") as f:
    for i in tqdm.tqdm(range(constant.shape[0])):
        f.write(str(float(constant[i])) + "\n")

100%|██████████| 44061/44061 [00:00<00:00, 60629.61it/s]


Тестовые файлы (проверка правильно угаданных нативных поз)

In [ ]:
newX_test = (A_inv.T @ X_test).T
print(newX.shape)

In [ ]:
with open("ki_test", "w") as f:
    for i in tqdm.tqdm(range(newX_test.shape[0])):
        y_i = ("+1 " if y_test[i] == 1 else "-1 ")
        f.write(y_i)
        for j in range(newX_test.shape[1]):
            f.write(str(j + 1) + ":" + str(newX_test[i,j]) + " ")
        f.write("\n")

### Строим модель в liblinear, достаем вектор $w$ из файла .model и тестируем модель.

In [61]:
with open("../kd_ki_train.model", "r") as f:
    data = f.read().split("\n")

In [62]:
newW = np.array(data[6:-1], dtype=float).reshape((6440, 1))

In [63]:
w = A_inv @ (newW + B)

In [66]:
np.savetxt("kd_ki_w.txt", w.T)

In [64]:
# Матрица признаков (для которых аффинности известны)
X_test = np.matrix([
    t[1][1]
    for t in test
]).T

# Столбец значений свободной энергии
s_test = np.matrix([
    float(t[0])
    for t in test
]).T

In [65]:
prediction = w.T @ X_test
print("Spearman: ", spearmanr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("Pearson: ", pearsonr(np.array(s_test.T)[0], np.array(prediction)[0]))
print("R2: ", r2_score(np.array(s_test.T)[0], np.array(prediction)[0]))
print("MSE: ", mean_squared_error(np.array(s_test.T)[0], np.array(prediction)[0]))

Spearman:  SpearmanrResult(correlation=0.66066142353770696, pvalue=1.1705380687619865e-194)
Pearson:  (0.65599302356610145, 5.1407873073395082e-191)
R2:  0.275077434998
MSE:  2.96949059399
